# TODO

- [ ] Code ran with flattened observation. How to run with nd-array observation? 
- [ ] How to provide pytorch net that I trained earlier for fronzen lake.
- [ ] Use ray tuner for hyperparamter tuning.

In [2]:
import gym
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray.rllib.agents.dqn import DQNTrainer

import pygame
import numpy as np

import torch
from IPython.display import clear_output
from IPython import display
import random
import matplotlib.pylab as plt
import copy
import time

from typing import List, Optional

In [3]:
# redifne this map to put starting and ending point in random location
# TODO: Can we do this with decorators?
def generate_random_map(size: int = 8, p: float = 0.8) -> List[str]:
    """Generates a random valid map (one that has a path from start to goal)
    Args:
        size: size of each side of the grid
        p: probability that a tile is frozen
    Returns:
        A random valid map
    """
    valid = False

    # DFS to check that it's a valid path.
    def is_valid(res,sx,sy):
        frontier, discovered = [], set()
        frontier.append((sx, sy))
        while frontier:
            r, c = frontier.pop()
            if not (r, c) in discovered:
                discovered.add((r, c))
                directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]
                for x, y in directions:
                    r_new = r + x
                    c_new = c + y
                    if r_new < 0 or r_new >= size or c_new < 0 or c_new >= size:
                        continue
                    if res[r_new][c_new] == "G":
                        return True
                    if res[r_new][c_new] != "H":
                        frontier.append((r_new, c_new))
        return False

    while not valid:
        p = min(1, p)
        res = np.random.choice(["F", "H"], (size, size), p=[p, 1 - p])
        sx = np.random.randint(size); sy = np.random.randint(size)
        res[sx][sy] = "S"
        gx = np.random.randint(size); gy = np.random.randint(size)
        while res[gx][gy] == "S": # we don't want to overwrite the S
            gx = np.random.randint(size); gy = np.random.randint(size)
        res[gx][gy] = "G"   
        valid = is_valid(res,sx,sy)
    return ["".join(x) for x in res]

In [50]:
class FrozenLakeWrapped(gym.Env):
    def __init__(self,config):
        self.size = 6 # grid size
        nH = 5.    # desired number of hole: this is probabilistic
        pH = 1 - nH/(self.size**2) # probability of  grid being a hole
        desc = generate_random_map(size= self.size, p= pH)
        self.env = gym.make('FrozenLake-v1', desc=desc, is_slippery=False)
        self.observation_space = gym.spaces.Box(0,1,shape=(3*self.size*self.size,), dtype=np.int32)
        self.action_space = self.env.action_space
    def oneHot(self,s):
        x = np.zeros(self.size*self.size)
        x[s] = 1
        state_ = np.array([ x.reshape(self.size,self.size),
                         np.array(self.env.desc == b"F").astype("int32"),
                         np.array(self.env.desc == b"G").astype("int32")
                          ])
        return state_.flatten()
    
    def reset(self):
        o = self.env.reset()
        return self.oneHot(o)

    def step(self, action):
        o, r, done, info = self.env.step(action)
        return self.oneHot(o), r, done, info
    
    def render(self):
        self.env.render()

In [51]:
env = FrozenLakeWrapped({})
env.reset()
done = False

while not done:
    # plt.cla()
    display.clear_output(wait=True)

    obs, reward, done, _ = env.step(env.action_space.sample())
    env.render()
    time.sleep(0.1)
    # display.clear_output(wait=True)
    # display.display(plt.gcf())
    # plt.gcf()

  (Down)
FFHSFF
FFFHFH
FFGFFF
FFHFFF
FFFFFF
FFFFFF


In [52]:
trainer = PPOTrainer(
    config={
        "env": FrozenLakeWrapped,
        "framework": "torch",
        "env_config": {},
        "num_workers": 4,
        
            "model": {
            "fcnet_hiddens": [256, 256],
            "fcnet_activation": "relu",
        },
    },
)

(RolloutWorker pid=28380) 2022-07-08 13:59:43,233	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=28381) 2022-07-08 13:59:43,228	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=28379) 2022-07-08 13:59:43,284	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=28378) 2022-07-08 13:59:43,278

In [53]:
for i in range(20):
    
    results = trainer.train()
    print(f"Iter: {i}; avg. reward={results['episode_reward_mean']}")


Iter: 0; avg. reward=0.33903133903133903
Iter: 1; avg. reward=0.4988009592326139
Iter: 2; avg. reward=0.6858168761220825
Iter: 3; avg. reward=0.7934936350777935
Iter: 4; avg. reward=0.8304696449026346
Iter: 5; avg. reward=0.9101851851851852
Iter: 6; avg. reward=0.9418493803622497
Iter: 7; avg. reward=0.9677152317880795
Iter: 8; avg. reward=0.9870722433460076
Iter: 9; avg. reward=0.9970501474926253
Iter: 10; avg. reward=0.9979064898813678
Iter: 11; avg. reward=0.9973579920739762
Iter: 12; avg. reward=0.9980569948186528
Iter: 13; avg. reward=1.0
Iter: 14; avg. reward=0.9993906154783668
Iter: 15; avg. reward=0.9993975903614458
Iter: 16; avg. reward=0.9994044073853484
Iter: 17; avg. reward=1.0
Iter: 18; avg. reward=0.998211091234347
Iter: 19; avg. reward=1.0


In [61]:
won = 0
lost = 0

for i in range(20):

    env = FrozenLakeWrapped({})
    # Get the initial observation (should be: [0.0] for the starting position).
    obs = env.reset()
    done = False
    total_reward = 0.0
    # Play one episode.
    while not done:
        # plt.cla()
        display.clear_output(wait=True)
        print("num won: ", won, " played: ", i)
        env.render()

        action = trainer.compute_single_action(obs)
        obs, reward, done, info = env.step(action)
        total_reward += reward
        time.sleep(0.1)
        
        # display.display(plt.gcf())
        # plt.gcf()
    if reward > 0 :
        won +=1 
    
        


num won:  2  played:  19
  (Right)
FFFFFF
FFFFFF
FFFFFF
HFFHHF
FFSFGF
FFFFHF
